In [1]:
import pickle
import os
import  numpy as np

In [2]:
def read(filename):
    with open(filename,"rb") as f:
        pkl_dict = pickle.load(f)
    return pkl_dict

In [3]:
distmult_dump = read('../symbolic/distmult_dump_norm.pkl')

In [4]:
distmult_dump.keys()

dict_keys(['tail_rel_type', 'entity_type', 'entity_to_id', 'head_rel_type', 'entity_real', 'rel_real', 'relation_to_id'])

In [5]:
def map_data(data,mapped_entity,mapped_relation):
    mapped_train_data=[]
    for line in data:
        mapped_train_data.append([mapped_entity[line[0]],mapped_relation[line[1]],mapped_entity[line[2]]])
    return mapped_train_data


In [6]:
def read_data(path):
    data = []
    with open(path,"r") as infile:
        lines = infile.readlines()
        data = [ line.strip().split() for line in lines]
    return data


In [7]:
def getRank(arr,elem):
    coun=0
    for val in arr:
        if(elem<val):
            coun+=1
    
    return coun

In [8]:
def analyze(table_dump,test_data):
    rv = []
    missing = 0
    for i,points in enumerate(test_data):
        key=(points[0],points[1])
        if(key in table_dump):
            table_dump[key] = np.array(table_dump[key])
            without_zeros=table_dump[key][table_dump[key] != 0]
            v,minv, maxv = (table_dump[key][points[2]], min(without_zeros), max(table_dump[key]))
            rank=getRank(without_zeros,v)
            if v > (minv + (maxv - minv)*.9):
                print(v,minv,maxv,rank,len(without_zeros))
                rv.append(i)
        else:
            #print('f****')
            missing += 1
        #
    return (rv,missing)
    
    
    

In [9]:
table_dump = read('./logs/13-10-2018-1430/3.pkl')

In [10]:
dataset_root='../data/fb15k'
test_data = read_data(os.path.join(dataset_root,"test.txt"))

In [14]:
test_data2=test_data[1:20]

In [15]:
test_data2i=map_data(test_data2,distmult_dump['entity_to_id'],distmult_dump['relation_to_id'])

In [16]:
explained,missing = analyze(table_dump['table'],test_data2i)

IndexError: too many indices for array

In [128]:
%debug


> <ipython-input-105-6ed894395013>(9)analyze()
      7             table_dump[key] = np.array(table_dump[key])
      8             without_zeros=table_dump[key][table_dump[key] != 0]
----> 9             v,minv, maxv = (table_dump[key][points[2]], min(without_zeros), max(table_dump[key]))
     10             rank=getRank(without_zeros,v)
     11             if v > (minv + (maxv - minv)*.9):

ipdb> key
(7161, 560)
ipdb> without_zeros
array([], dtype=int64)
ipdb> i
1045
ipdb> exit


In [129]:
test_data2[1045]


['/m/026sdt1',
 '/film/film_job/films_with_this_crew_job./film/film_crew_gig/crewmember',
 '/m/0gl88b']

In [64]:
a= np.array(a)


In [65]:
a

array([1, 2])

array([1, 2])

In [17]:
print(table_dump['table'].keys())

dict_keys([(0, 0), (2, 1), (4, 2), (6, 3), (8, 4), (10, 5), (12, 6), (14, 7), (16, 8), (18, 9), (20, 10), (22, 11), (24, 12), (26, 13), (28, 14), (30, 15), (32, 16), (34, 17), (36, 18), (38, 19), (40, 20), (42, 21), (44, 13), (46, 22), (48, 23), (50, 24), (52, 25), (54, 26), (56, 27), (58, 28), (60, 9), (62, 29), (63, 13), (65, 11), (67, 30), (69, 31), (71, 32), (73, 33), (75, 34), (77, 35), (79, 36), (81, 37), (83, 38), (85, 39), (87, 38), (89, 40), (91, 7), (93, 20), (94, 7), (96, 41), (97, 42), (98, 30), (100, 43), (102, 44), (104, 45), (106, 46), (108, 47), (110, 20), (112, 11), (114, 48), (115, 9), (117, 49), (119, 50), (121, 48), (122, 13), (124, 51), (126, 37), (128, 52), (130, 53), (131, 3), (133, 10), (135, 54), (137, 55), (139, 49), (141, 56), (142, 57), (143, 58), (145, 59), (147, 60), (149, 61), (151, 9), (153, 62), (155, 3), (157, 9), (159, 43), (160, 50), (162, 63), (164, 13), (167, 64), (169, 45), (171, 53), (173, 4), (175, 48), (177, 43), (178, 44), (180, 65), (182, 66)

In [20]:
print(table_dump['table'][(2,1)])

{3: (1.0000001, 1), 58: (0.0087936325, 422), 2612: (0.45938587, 470), 8729: (0.27344, 398), 11377: (1.0000001, 1)}
